# Labs - Biopython and data formats

## Outline

- Managing dependencies in Python with environments
- Biopython 
    - Sequences (parsing, representation, manipulation)
    - Structures (parsing, representation, manipulation)

### 1. Python environments

- handles issues with dependencies versions
- ensures reproducibility
- does not clutter users' global site-packages directory

`python3 -m venv venv/       # Creates an environment called venv/`
`source venv/bin/activate`
`pip install biopython`
`pip freeze > requirements.txt`
`(venv) % deactivate`

On a different machine, the environment can be replicated by creating a new environment and running

`pip install -r requirements.txt`

### 2. Biopython

Biopython is a library consisting of tools for both sequence and structure bioinformatics. Among other things it enables parsing, handling and storing molecular data present in common formats such as FASTA, PDB or mmCIF.

Install biopython using `pip install biopython`

Functionality divided into packages list of which is available in the [docs](https://biopython.org/docs/1.75/api/Bio.html). 

Main sequence and structure packages:
 - [Bio.Seq](https://biopython.org/docs/latest/api/Bio.Seq.html)
 - [Bio.Align](https://biopython.org/docs/latest/api/Bio.Align.html) 
 - [Bio.SeqIO](https://biopython.org/docs/latest/api/Bio.SeqIO.html)
 - [Bio.PDB](https://biopython.org/docs/latest/api/Bio.PDB.html) 
 

 
#### Sequences 
 
 Loading a sequence from a string: 

In [3]:
from Bio.Seq import Seq
seq = Seq("AGTACACTG")
print(seq)

AGTACACTG


This creates a [sequence object](https://biopython.org/docs/latest/api/Bio.Seq.html) with a couple of fancy methods, especially when it comes to nuclotide sequences such as `reverse_complement` or `translate`.

In [4]:
print(seq.translate())
print(seq.reverse_complement().transcribe())
print(seq.reverse_complement().transcribe().translate())

STL
CAGUGUACU
QCT


In [5]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
print(coding_dna.translate())
print(coding_dna.translate(to_stop=True))
print(coding_dna.translate(table=2))
print(coding_dna.translate(table=2, to_stop=True))

MAIVMGR*KGAR*
MAIVMGR
MAIVMGRWKGAR*
MAIVMGRWKGAR


Notice, in the example above we used different genetic tables. Check [NCBI genetic codes](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi) for details.

To list all the methods, run, e.g., one of the following:

In [59]:
print(dir(seq))
print(help(seq))

['__abstractmethods__', '__add__', '__bytes__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__imul__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_data', 'back_transcribe', 'complement', 'complement_rna', 'count', 'count_overlap', 'encode', 'endswith', 'find', 'index', 'join', 'lower', 'lstrip', 'replace', 'reverse_complement', 'reverse_complement_rna', 'rfind', 'rindex', 'rsplit', 'rstrip', 'split', 'startswith', 'strip', 'tomutable', 'transcribe', 'translate', 'ungap', 'upper']
b'AGTACACTG'


Methods for accessing by position  are available as well.

In [60]:
print(seq[3])
print(seq[3:5])
print(seq[::-1])

A
AC
GTCACATGA


If needed, the Seq object can be converted into a string.

In [3]:
print(str(seq))
print(str(seq).translate({65: 88}))

AGTACACTG
XGTXCXCTG


To parse sequence from a file, you can use [Bio.SeqIO](https://biopython.org/docs/latest/api/Bio.SeqIO.html). [Here](https://biopython.org/wiki/SeqIO#file-formats) is the list of supported formats. The format name is passed into the `parse` method.

In [7]:
from Bio import SeqIO

sars2_it = SeqIO.parse("R1A-B_SARS2.fasta", "fasta")
for seq_record in sars2_it:    
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))
sars2_seq_recs = list(sars2_it)

sp|P0DTD1|R1AB_SARS2
Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...VNN')
7096
sp|P0DTC1|R1A_SARS2
Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...FAV')
4405


The result is an iterator of [SeqRecord](https://biopython.org/docs/latest/api/Bio.SeqRecord.html)s. Other attributes of `SeqRecord` such as features or annotations are more relevant for other formats, such as genbank. The underlying gene for the two isoforms (R1A_SARS2/P0DTC1 and R1AB_SARS2/P0DTD1) is ORF1ab and the two isoforms are caused by ribosomal slippage during translation (see, e.g., [here](https://www.science.org/doi/full/10.1126/science.abf3546)). Both reading frames R1A_SARS2 and R1AB_SARS2 are polyproteins and are encoded by the same [gene](https://www.ncbi.nlm.nih.gov/gene/43740578). Let's explore this.

In [9]:
gb_rec = list(SeqIO.parse("NC_045512.gb", "genbank"))[0]
print(gb_rec.)

NC_045512.2


In [10]:
print(gb_rec.annotations)
print(gb_rec.features)

{'molecule_type': 'ss-RNA', 'topology': 'linear', 'data_file_division': 'VRL', 'date': '18-JUL-2020', 'accessions': ['NC_045512'], 'sequence_version': 2, 'keywords': ['RefSeq'], 'source': 'Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)', 'organism': 'Severe acute respiratory syndrome coronavirus 2', 'taxonomy': ['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus'], 'references': [Reference(title='A new coronavirus associated with human respiratory disease in China', ...), Reference(title='Programmed ribosomal frameshifting in decoding the SARS-CoV genome', ...), Reference(title='The structure of a rigorously conserved RNA element within the SARS virus genome', ...), Reference(title="A phylogenetically conserved hairpin-type 3' untranslated region pseudoknot functions in coronavirus RNA replication", ...), Reference(title='Direct Submission', .

In [13]:
gb_rec.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(29903), strand=1), type='source'),
 SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(265), strand=1), type="5'UTR"),
 SeqFeature(FeatureLocation(ExactPosition(265), ExactPosition(21555), strand=1), type='gene'),
 SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(265), ExactPosition(13468), strand=1), FeatureLocation(ExactPosition(13467), ExactPosition(21555), strand=1)], 'join'), type='CDS', location_operator='join'),
 SeqFeature(FeatureLocation(ExactPosition(265), ExactPosition(805), strand=1), type='mat_peptide'),
 SeqFeature(FeatureLocation(ExactPosition(805), ExactPosition(2719), strand=1), type='mat_peptide'),
 SeqFeature(FeatureLocation(ExactPosition(2719), ExactPosition(8554), strand=1), type='mat_peptide'),
 SeqFeature(FeatureLocation(ExactPosition(8554), ExactPosition(10054), strand=1), type='mat_peptide'),
 SeqFeature(FeatureLocation(ExactPosition(10054), ExactPosition(10972), strand=1), type='mat_

Let's obtain all CDS (coding sequence) features.

In [16]:
print(gb_rec.features)
cds = [seq_feature for seq_feature in gb_rec.features if seq_feature.type == 'CDS']

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(29903), strand=1), type='source'), SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(265), strand=1), type="5'UTR"), SeqFeature(FeatureLocation(ExactPosition(265), ExactPosition(21555), strand=1), type='gene'), SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(265), ExactPosition(13468), strand=1), FeatureLocation(ExactPosition(13467), ExactPosition(21555), strand=1)], 'join'), type='CDS', location_operator='join'), SeqFeature(FeatureLocation(ExactPosition(265), ExactPosition(805), strand=1), type='mat_peptide'), SeqFeature(FeatureLocation(ExactPosition(805), ExactPosition(2719), strand=1), type='mat_peptide'), SeqFeature(FeatureLocation(ExactPosition(2719), ExactPosition(8554), strand=1), type='mat_peptide'), SeqFeature(FeatureLocation(ExactPosition(8554), ExactPosition(10054), strand=1), type='mat_peptide'), SeqFeature(FeatureLocation(ExactPosition(10054), ExactPosition(10972), strand=1), type='mat_peptide'

In [19]:
cds[0].extract(gb_rec.seq).translate()

Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...NN*')

Now, let's get the DNA sequence for the the polyprotein 1ab.

In [22]:
aa_seq = cds[0].extract(gb_rec.seq).translate()
print(aa_seq[-10:])
print(gb_rec.seq.translate()[-10:])

ISSDVLVNN*
KKKKKKKKKK


To write a sequence into a file, use `SeqIO.write`.

In [23]:
SeqIO.write([gb_rec, SeqIO.SeqRecord(aa_seq, id="id", description="aa")], "fasta_from_gb.fasta", "fasta")

2

Now, carry out the following tasks by yourselfs:
- Obtain the protein sequnece for polyprotein 1ab and check with UniProt that it matches (just by eyeballing).
- Obtain the protein sequence for the polyprotien 1a.
- Obtain protein sequences for all the proteins and list them together with their names


In [37]:
print(cds[1].extract(gb_rec.seq).translate())

MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQTT

In [25]:
#print(cds[1].extract(gb_rec.seq).translate())

peptides = [print("{}: {}".format(ft.qualifiers['protein_id'], ft.extract(gb_rec.seq).translate())) for ft in gb_rec.features if ft.type == 'mat_peptide']


['YP_009725297.1']: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGG
['YP_009725298.1']: AYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGG
['YP_009725299.1']: APTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEE

#### Structures
Structure processing is managed by the [Bio.PDB](https://biopython.org/docs/latest/api/Bio.PDB.html) package.

To read a structure from a PDB file, use the `PDBParser`. We will be using the 3C-like proteinase protein, which is one of the processed proteins present in the ORF1a discussed above. One of it's structures is [7ALH](https://www.ebi.ac.uk/pdbe/entry/pdb/7alh). To see all the proteins, I suggest checking out the PDBe-KB page for [P0DTD1](https://www.ebi.ac.uk/pdbe/pdbe-kb/protein/P0DTD1).

In [103]:
from Bio.PDB.PDBParser import PDBParser
parser = PDBParser(PERMISSIVE=1)
structure = parser.get_structure("7alh", "7alh.ent")

As the PDB format is considered deprecated, one should use the mmCIF file instead. This is done the same way as in case of PDB files.

In [27]:
from Bio.PDB.MMCIFParser import MMCIFParser
parser = MMCIFParser()
structure = parser.get_structure("7alh", "7alh.cif")

To retrieve the individual CIF dictionary fields, one can use the `MMCIF2Dict` module.

In [26]:
from Bio.PDB.MMCIFParser import MMCIF2Dict
mmcif_dict = MMCIF2Dict("7alh.cif")
print(mmcif_dict["_citation.title"])

['Crystal structure of the main protease (3CLpro/Mpro) of SARS-CoV-2 at 1.65A resolution (spacegroup C2).']


The structure record has the structure->model->chain->residue->architecture.

![SMRCA](http://biopython.org/DIST/docs/tutorial/images/smcra.png)

Each of the levels in the hierarchy is represented by a submodule in Bio.PDB, namely [Bio.Structure](https://biopython.org/docs/latest/api/Bio.PDB.Modul.html), [Bio.Module](https://biopython.org/docs/latest/api/Bio.PDB.Module.html),[Bio.Chain](https://biopython.org/docs/latest/api/Bio.PDB.Chain.html),[Bio.Residue](https://biopython.org/docs/latest/api/Bio.PDB.Residue.html) and [Bio.Atom](https://biopython.org/docs/latest/api/Bio.PDB.Atom.html). For details regarding IDs, check the [section on ID](https://biopython.org/docs/1.75/api/Bio.PDB.Entity.html#Bio.PDB.Entity.Entity.get_full_id) of the Entity class which is the superclass of the Module/Chain/Residue/Atom classes.

In [150]:
print(structure.get_list())

print('---------- MODEL INFO ----------')

model = structure[0]
print("Full ID: {}\nID: {}".format(model.get_full_id(), model.get_id()))
print(model.get_list())

print('---------- CHAIN INFO ----------')
chain = model['A']
print("Full ID: {}\nID: {}".format(chain.get_full_id(), chain.get_id()))
print(chain.get_list())

print('---------- RESIDUE INFO ----------')
res = chain[(' ',1,' ')]
print("Full ID: {}\nID: {}".format(res.get_full_id(), res.get_id()))
print(res.get_resname())
res = chain[1]
print(res.get_resname())

print(res.get_list())
print('---------- ATOM INFO ----------')
atom=res['CA']
print("Full ID: {}\nID: {}".format(atom.get_full_id(), atom.get_id()))
print("{}\n{}\n{}\n{}".format(atom.get_name(), atom.get_id(), atom.get_coord(), atom.get_fullname()))
print(atom.get_vector())


[<Model id=0>]
---------- MODEL INFO ----------
Full ID: ('7alh', 0)
ID: 0
[<Chain id=A>]
---------- CHAIN INFO ----------
Full ID: ('7alh', 0, 'A')
ID: A
[<Residue SER het=  resseq=1 icode= >, <Residue GLY het=  resseq=2 icode= >, <Residue PHE het=  resseq=3 icode= >, <Residue ARG het=  resseq=4 icode= >, <Residue LYS het=  resseq=5 icode= >, <Residue MET het=  resseq=6 icode= >, <Residue ALA het=  resseq=7 icode= >, <Residue PHE het=  resseq=8 icode= >, <Residue PRO het=  resseq=9 icode= >, <Residue SER het=  resseq=10 icode= >, <Residue GLY het=  resseq=11 icode= >, <Residue LYS het=  resseq=12 icode= >, <Residue VAL het=  resseq=13 icode= >, <Residue GLU het=  resseq=14 icode= >, <Residue GLY het=  resseq=15 icode= >, <Residue CYS het=  resseq=16 icode= >, <Residue MET het=  resseq=17 icode= >, <Residue VAL het=  resseq=18 icode= >, <Residue GLN het=  resseq=19 icode= >, <Residue VAL het=  resseq=20 icode= >, <Residue THR het=  resseq=21 icode= >, <Residue CYS het=  resseq=22 icode

To download a file from PDB, one can use the PDBList module.

In [28]:
from Bio.PDB.PDBList import PDBList
pdbl = PDBList()
pbl_7lkr=pdbl.retrieve_pdb_file("7LKR", file_format="mmCif", pdir=".")

Structure exists: '.\7lkr.cif' 


In [29]:
from Bio.PDB.MMCIFParser import MMCIFParser
parser = MMCIFParser()
structure = parser.get_structure("7lkr", "7lkr.cif")

c:\python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4467.
  warnings.warn(
c:\python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4531.
  warnings.warn(
c:\python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4608.
  warnings.warn(
c:\python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4774.
  warnings.warn(


Tasks:
- Iterate over all atoms of the structure
- List all water residues (the first field of the residue id is 'W')
- How many water molecules is in the recrod?
- How many heteroatoms are there in the recod (the first field of the residue id is 'H').
- Find a structure in PDB with at least one ligand (different from water) and write a code which lists all the ligands.